# Test AVI Files

### a. AVI with 4-Class Fragment Mask Colors

In [ ]:
import cv2

fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_out = cv2.VideoWriter(projectPath+'results/seeding_efficiency/segmentation.avi',
                            fourcc, 3, (width, height))

# Fragment color map (BGR)
colors = [
    (230, 25, 75),    # Class 0: Apoptotic  - Red
    (67, 99, 216),    # Class 1: Necrotic   - Blue
    (60, 180, 75),    # Class 2: Fixed      - Green
    (245, 130, 48),   # Class 3: Regular    - Orange
]

for i, result in enumerate(results):
  masks = result['masks']
  image = result['image']
  props = regionprops(label(masks), intensity_image=image)

  vis = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

  for p in props:
      area = p.area
      ecc = p.eccentricity
      sol = p.solidity

      # Fragment classification rule
      if area < 40 and ecc > 0.9:       # Small and elongated -> apoptotic
        cls = 0
      elif sol < 0.75:                  # very irregular shape -> necrotic
        cls = 1
      elif ecc > 0.9:                   # elongated -> possibly fixed
        cls = 2
      else:                             # mostly round and regular
        cls = 3

      # Draw pixels belonging to this region in class color
      coords = p.coords
      for y, x in coords:
          vis[y, x] = colors[cls]

  video_out.write(vis)

video_out.release()


### b. AVI with Overlay of Masks + Original Image

In [ ]:
video_overlay = cv2.VideoWriter(projectPath+'results/seeding_efficiency/overlay.avi',
                                fourcc, 3, (width, height))

for i, result in enumerate(results):
  masks = result['masks']
  image = result['image']

  if image.ndim == 2:  # grayscale → convert to RGB
      img_rgb = np.stack([image] * 3, axis=-1)
  else:
      img_rgb = image.copy()

  img_rgb = (img_rgb / img_rgb.max() * 255).astype(np.uint8)

  overlay = img_rgb.copy()

  props = regionprops(label(masks), intensity_image=image)
  for p in props:
      area = p.area
      ecc = p.eccentricity
      sol = p.solidity

      # Fragment classification rule
      if area < 40 and ecc > 0.9:       # Small and elongated -> apoptotic
        cls = 0
      elif sol < 0.75:                  # very irregular shape -> necrotic
        cls = 1
      elif ecc > 0.9:                   # elongated -> possibly fixed
        cls = 2
      else:                             # mostly round and regular
        cls = 3

      coords = p.coords
      for y, x in coords:
          overlay[y, x] = colors[cls]

  video_overlay.write(overlay)

video_overlay.release()